## Illustrate photon noise

Generate a video illustrating the impact of photon noise on a simulated image.

Note: This script is slow to run.

In [ ]:
# Default import
import sys
sys.path.append('../')
from helpers import create_figure, show_image, glue_fig, load_image

import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import *

fig, (ax_im, ax_hist, ax_snr) = plt.subplots(1, 3, figsize=(12, 4), dpi=200)
    
seed = 1012
rand = np.random.default_rng(seed=seed)

im = load_image('happy_cell.tif')
im = im.astype(np.float32)
im = im / np.percentile(im, 99)
im = np.maximum(im, 0.3) # Want a non-zero background
mean = im.mean()

im_output = np.zeros(im.shape, dtype=im.dtype)

fps = 5
writer = FFMpegWriter(fps=fps)
name = '../_static/videos/photon_noise.mp4'

# writer = HTMLWriter(fps=fps)
# name = 'myfile.html'

# writer = PillowWriter(fps=fps)
# name = 'myfile.gif'

max_time = 1000
n = 51
rate = max_time / n / 10

with writer.saving(fig, name, dpi=200):
    time = np.linspace(0, 1000, n)
    for ii, t in enumerate(time):
        if ii > 0:
            # Don't add for the first entry
            im_output += rand.poisson(im * rate)
        
        m = np.percentile(im_output, 95)
        ax_im.clear()
        ax_im.imshow(im_output, vmin=0, vmax=m, animated=True, cmap='gray')
        ax_im.text(10, 20, f'{round(t)} ms', color=(1,1,1), fontdict={'size': 14})
        ax_im.set_axis_off()

        ax_hist.clear()
        hist_max = int(np.ceil(mean * n * rate * 4))
        ax_hist.hist(im_output.flatten(), bins=hist_max, density=True, range=(0, hist_max))
        ax_hist.set_xlim(0, hist_max)
        ax_hist.set_xlabel('Pixel value')
        ax_hist.set_ylabel('Frequency')
        
        ax_snr.clear()
        ax_snr.plot(time[:ii], np.sqrt(time[:ii]), scalex=False, scaley=False)
        ax_snr.set_xlim(0, max_time)
        ax_snr.set_ylim(0, np.ceil(np.sqrt(max_time)))
        ax_snr.set_xlabel('Time (ms)')
        ax_snr.set_ylabel('SNR')
        ax_snr.set_yticks([]) # Since this is an illustration & we haven't tried to quantify SNR any more exactly

        if ii == 0:
            plt.tight_layout()
            plt.subplots_adjust(wspace=0.3)
        
        writer.grab_frame()